In [36]:
import numpy as np
import pandas as pd
import random
import cv2
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D 
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow import keras
# from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers.legacy import SGD
from tensorflow.keras import backend as k
import os

In [34]:
def  sum_scaled_weights(scaled_weight_list):
    '''Returns the sum of the listed scaled weights.
    scaled average of the weights'''
    avg_grad = list()
    #  get the average grad over all client gradients
    for grad_list_tupel in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tupel, axis=0)
        avg_grad.append(layer_mean)
    return avg_grad


In [42]:
def scale_model_weights(weight,scalar):
    '''Function for scaling a model's weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar*weight[i])
    return weight_final

In [22]:
class SimpleMLP:
    @staticmethod
    def build(shape, classes):
        model = Sequential()
        model.add(Dense(200, input_shape=(shape,)))
        model.add(Activation("relu"))
        model.add(Dense(200))
        model.add(Activation("relu"))
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        return model
lr = 0.01
coms_round = 100
loss = 'categorical_crossentropy'
metrics = ['accuracy']
optimizer = SGD(learning_rate = lr,
                momentum = 0.9
                )

In [43]:
# Read all the h5 files from the path
img_path = '/home/nextg3/Documents/Thesis/Code/FL/local_models'
# Get path list
image_paths = list(paths.list_files(img_path))
local_models  = []

# get all models
for p in image_paths:
    test_model = SimpleMLP()
    local_model = test_model.build(784,10)
    local_model.compile(loss=loss,
                        optimizer=optimizer,
                        metrics=metrics)
    # set weights of the global model as weights of the local modelk

    local_model.load_weights(p)
    scaled_model_weights = scale_model_weights(local_model.get_weights(),len(image_paths))
    local_models.append(scaled_model_weights)
    k.clear_session()



# Final weights
average_weights = sum_scaled_weights(local_models)
smlp_global = SimpleMLP()
global_model = smlp_global.build(784,10)
global_model.set_weights(average_weights)
global_model.save_weights('/home/nextg3/Documents/Thesis/Code/FL/global_model.h5')

# # update global model
# global_model.set_weights(average_weights)

In [30]:
# img_path = '/home/nextg3/Documents/Thesis/Code/FL/local_models'
# list(paths.list_files(img_path))
len(local_models)

10

In [27]:
test_model = SimpleMLP()
local_model = test_model.build(784,10)
local_model.compile(loss=loss,
                    optimizer=optimizer,
                    metrics=metrics)
# set weights of the global model as weights of the local modelk
local_model.load_weights('/home/nextg3/Documents/Thesis/Code/FL/weights.h5')

In [26]:
k.clear_session()